In [1]:
import torch.nn.utils.prune as prune
import torch
from torchvision.models import resnet18
import torch_pruning as pruning
from torchsummary import summary
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = torch.load('vgg/vgg11.pth').to(device)
# build layer dependency for resnet18
model.eval()
# prune.random_unstructured(model.conv1, name="weight", amount=0.3)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [2]:
model.features[0].weight

Parameter containing:
tensor([[[[ 0.2882,  0.0358, -0.3850],
          [ 0.1795,  0.3668, -0.5012],
          [-0.0974,  0.3648, -0.2296]],

         [[ 0.4015, -0.0461, -0.6842],
          [ 0.4442,  0.4478, -0.7949],
          [ 0.1129,  0.4917, -0.3705]],

         [[ 0.2162, -0.0290, -0.3949],
          [ 0.1490,  0.2967, -0.4294],
          [-0.0095,  0.3479, -0.1558]]],


        [[[-0.3528, -0.2549,  0.6734],
          [-0.6027, -0.3453,  0.8054],
          [-0.4197, -0.1347,  0.6622]],

         [[-0.5740, -0.3998,  0.7708],
          [-0.8758, -0.3668,  1.1098],
          [-0.5186, -0.0801,  0.9228]],

         [[-0.0280, -0.2954,  0.2159],
          [-0.1868, -0.2904,  0.3808],
          [-0.0926, -0.0766,  0.3727]]],


        [[[ 0.0650, -0.2592, -0.2389],
          [ 0.3209,  0.2152, -0.2047],
          [-0.0443,  0.2227, -0.0452]],

         [[ 0.0290, -0.4797, -0.5490],
          [ 0.5363,  0.2973, -0.3728],
          [ 0.1776,  0.4145, -0.0436]],

         [[ 0.0033, -0

In [3]:
summary(model, (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,792
              ReLU-2           [-1, 64, 64, 64]               0
         MaxPool2d-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 32, 32]          73,856
              ReLU-5          [-1, 128, 32, 32]               0
         MaxPool2d-6          [-1, 128, 16, 16]               0
            Conv2d-7          [-1, 256, 16, 16]         295,168
              ReLU-8          [-1, 256, 16, 16]               0
            Conv2d-9          [-1, 256, 16, 16]         590,080
             ReLU-10          [-1, 256, 16, 16]               0
        MaxPool2d-11            [-1, 256, 8, 8]               0
           Conv2d-12            [-1, 512, 8, 8]       1,180,160
             ReLU-13            [-1, 512, 8, 8]               0
           Conv2d-14            [-1, 51

In [4]:
# parameters_to_prune = (
#     (model.classifier[0], 'weight'),
#     # (model.classifier[3], 'weight'),

# )

# prune.global_unstructured(
#     parameters_to_prune,
#     pruning_method=prune.RandomUnstructured,
#     amount=0.2,
# )
# print(prune.is_pruned(model))
# for each_parameter in parameters_to_prune:
#     prune.remove(each_parameter[0], each_parameter[1])

# summary(model, (3, 64, 64))

In [5]:
parameters_to_prune = (
    (model.features[0], 'weight'),
    (model.features[3], 'weight'),
    (model.features[6], 'weight'),
    (model.features[8], 'weight'),
    (model.features[11], 'weight'),
    (model.features[13], 'weight'),
    (model.features[16], 'weight'),
    (model.features[18], 'weight'),
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)
print(prune.is_pruned(model))
for each_parameter in parameters_to_prune:
    prune.remove(each_parameter[0], each_parameter[1])

True


In [10]:
summary(model, (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,792
              ReLU-2           [-1, 64, 64, 64]               0
         MaxPool2d-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 32, 32]          73,856
              ReLU-5          [-1, 128, 32, 32]               0
         MaxPool2d-6          [-1, 128, 16, 16]               0
            Conv2d-7          [-1, 256, 16, 16]         295,168
              ReLU-8          [-1, 256, 16, 16]               0
            Conv2d-9          [-1, 256, 16, 16]         590,080
             ReLU-10          [-1, 256, 16, 16]               0
        MaxPool2d-11            [-1, 256, 8, 8]               0
           Conv2d-12            [-1, 512, 8, 8]       1,180,160
             ReLU-13            [-1, 512, 8, 8]               0
           Conv2d-14            [-1, 51

In [6]:
list(model.features[0].named_parameters())

[('bias',
  Parameter containing:
  tensor([ 0.1939,  0.3042,  0.1825, -1.1122,  0.0442, -0.0678,  0.1324, -0.5846,
           0.2210, -0.0130, -0.3794,  0.1256,  0.2415,  0.2491, -0.7849, -1.0575,
           0.2637, -0.1838,  0.1533, -1.1485,  0.0095, -0.8640, -0.3903, -0.4040,
           0.5775,  0.2528,  0.0911,  0.1554, -0.1833, -0.4766, -0.2352, -1.1038,
          -0.5924,  0.2940,  0.1892,  0.4142, -0.0220,  0.1800, -0.5532,  0.2365,
           0.1548, -1.7165, -0.0318,  0.0494,  0.1286,  0.1860,  0.1789,  0.4679,
          -0.0066, -0.0948, -1.4731, -0.7618, -1.2010,  0.1765,  0.2015, -0.0822,
           0.1453,  0.0289, -1.2024,  0.1595, -0.8845, -0.0075,  0.2292, -1.3837],
         device='cuda:0', requires_grad=True)),
 ('weight',
  Parameter containing:
  tensor([[[[ 0.2882,  0.0358, -0.3850],
            [ 0.1795,  0.3668, -0.5012],
            [-0.0974,  0.3648, -0.2296]],
  
           [[ 0.4015, -0.0461, -0.6842],
            [ 0.4442,  0.4478, -0.7949],
            [ 0.

In [7]:
print(
    "Sparsity in features[0]: {:.2f}%".format(
        100. * float(torch.sum(model.features[0].weight == 0))
        / float(model.features[0].weight.nelement())
    )
)
print(
    "Sparsity in features[3].weight: {:.2f}%".format(
        100. * float(torch.sum(model.features[3].weight == 0))
        / float(model.features[3].weight.nelement())
    )
)
# print(
#     "Sparsity in fc1.weight: {:.2f}%".format(
#         100. * float(torch.sum(model.fc1.weight == 0))
#         / float(model.fc1.weight.nelement())
#     )
# )
# print(
#     "Sparsity in fc2.weight: {:.2f}%".format(
#         100. * float(torch.sum(model.fc2.weight == 0))
#         / float(model.fc2.weight.nelement())
#     )
# )
# print(
#     "Sparsity in fc3.weight: {:.2f}%".format(
#         100. * float(torch.sum(model.fc3.weight == 0))
#         / float(model.fc3.weight.nelement())
#     )
# )
# print(
#     "Global sparsity: {:.2f}%".format(
#         100. * float(
#             torch.sum(model.conv1.weight == 0)
#             + torch.sum(model.conv2.weight == 0)
#             + torch.sum(model.fc1.weight == 0)
#             + torch.sum(model.fc2.weight == 0)
#             + torch.sum(model.fc3.weight == 0)
#         )
#         / float(
#             model.conv1.weight.nelement()
#             + model.conv2.weight.nelement()
#             + model.fc1.weight.nelement()
#             + model.fc2.weight.nelement()
#             + model.fc3.weight.nelement()
#         )
#     )
# )

Sparsity in features[0]: 1.45%
Sparsity in features[3].weight: 10.11%


In [8]:
torch.save(model, 'vgg11_pruning_pytorch.pth')

In [9]:
class FooBarPruningMethod(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        mask.view(-1)[::2] = 0
        return mask
